In [ ]:
# default_exp data
%load_ext nb_black
# nb_black if running in jupyter
# lab_black
%load_ext autoreload
# automatically reload python modules if there are changes in the
%autoreload 2

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


<IPython.core.display.Javascript object>

In [ ]:
# hide
from nbdev.showdoc import *

<IPython.core.display.Javascript object>

# Data

> You should begin your work by cleaning up your data and possibly defining tools for doing it repeateadly.



***input***: raw data

***output***: clean and tidy dataset for ML model / simulation / analytics + toy dataset for testing

***description:***

This is the first notebook of your machine learning project. In this notebook, you will load the data, inspect, clean and make it tidy. 
You will define the data points and their features and labels. The output of this notebook is a clean, tidy dataset ready for analysis and machine learning.
You can also do a basic statistical analysis of the data to better understand it.
For any functions you define for handling the data, remember to mark their cells with `# export` -comment,
so that they will be included in the data.py-module build based on this notebook.
You can also include unit tests for your own functions.

## Import relevant modules

Import python modules you need for handling the data

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from numpy import nan as Nan
import pandas as pd
from sklearn import datasets

# from pandas.api.types import CategoricalDtype

<IPython.core.display.Javascript object>

## Define notebook parameters

Define input, output and additional parameters of this notebook, the information needed for running the notebook.
In your own project, you can do this step in the later iterations of the work,
when you know what is required.
In this cell, only assing values to variables directly: `variable_name = value`.
**Do not derive any information in this cell as it will mess up the parameterization** - do it in the cell below.

The cell below has the tag 'parameters' - this is for the notebook parameterization tool 'papermill'
that allows you execute complete notebooks as python functions.
The values you define here will become the default values of the parameterized notebook, but you can also run the notebook with completely different setup.
More on this in the `workflow` notebook.

In [ ]:
# This cell is tagged with 'parameters'
output_clean_filepath = (
    "data/preprocessed_data/dataset_clean_hki_lib_book_classification.csv"
)
output_toy_filepath = (
    "data/preprocessed_data/dataset_toy_hki_lib_book_classification.csv"
)
seed = 0

<IPython.core.display.Javascript object>

Now we can define any immediate derivative operations, righ below the parameters cell:

In [ ]:
# set seed
np.random.seed(seed)

<IPython.core.display.Javascript object>

## Load the data

We are using a csv-file exported from database of Helsinki City library system. We could also load the data straight from database or from online resources, if needed and possible. Please note, that you should not add your datasets to git, as it is not intended for data version control and tracking large datafiles exceed the limits of it. The 'data'-folder of this template is ignored by git.

In [ ]:
#
# Read the raw data of library volumes. The fields are separated by commas.
#

df_raw_data = pd.read_csv("data/raw_data/sample4.csv")

<IPython.core.display.Javascript object>

## Describe the data

The database uses so called MARC-system. There are multiple rows for each volume(=item). The file consists of three columns:

id (id of the Volume that is item e.g. a book)
content (the metadata belonging to this volume)
marc_tag (specifies what the value of the 'content'-field in this row means), for example:
marc_tag '650' means that content field holds a keyword
marc_tag '095' means that content field holds Helsinki-classification

In [ ]:
df_raw_data.shape

(820271, 7)

<IPython.core.display.Javascript object>

In [ ]:
# view the data
df_raw_data.head()

,record_id,084,092,093,094,095,650
0,420907795010,NaN,NaN,NaN,NaN,180.2,NaN
1,420907795011,NaN,NaN,NaN,1.4,NaN,NaN
2,420907795013,NaN,NaN,NaN,1.4,NaN,NaN
3,420907795014,NaN,NaN,NaN,NaN,1.791,NaN
4,420907795016,NaN,1.4,NaN,1.4,NaN,NaN


<IPython.core.display.Javascript object>

In [ ]:
df_raw_data.info()

# An example in dataset 'Sample4.csv, where there is no HKI-CLASS.
# Instead the row has 084- and 092 Classes and some keywords

print("\n\nNO HKI CLASS:\n", df_raw_data.loc[df_raw_data["record_id"] == 420907796975])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 820271 entries, 0 to 820270
Data columns (total 7 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   record_id  820271 non-null  int64 
 1   084        271200 non-null  object
 2   092        428182 non-null  object
 3   093        157261 non-null  object
 4   094        422208 non-null  object
 5   095        552728 non-null  object
 6   650        664145 non-null  object
dtypes: int64(1), object(6)
memory usage: 43.8+ MB


NO HKI CLASS:
         record_id    084    092  093    094  095  \
875  420907796975  68.25  68.25  NaN  68.25  NaN   

                                                   650  
875  ruokaohjeet,ruokaperinne,ruokakulttuuri,arktin...  


<IPython.core.display.Javascript object>

Ok, so there are 13 features and one label (the last column in our dataset) in the data. Let's construct these into dataframe column names.


## Clean the data and make it tidy

For analytics & ML purposes, we should make the data tidy. This means that

 1. Every column is a variable (feature or label)
 2. Every row is an observation (data point).
 3. Every cell is a single value (int, float, cathegorical, str, but no nested structures like lists or dictionaries)


 - Our **features** consist of keywords and different classifications. Marc_tags for this features are:
   - 084 = PLC - Finnish Public Libraries Classification System (YKL)
   - 092 = City of Espoo classification
   - 093 = City of Kauniainen classification
   - 094 = City of Vantaa classification
   - 650 = Keyword (subject/index term/concept)
 - Our **label** is Helsinki City Library Classification System and we call it HCLCS (HKLJ) (095 in raw data).
 - Our **datapoint** is a library item (book, cd, dvd...) identified by id column in raw data.
 
~~First we get only those rows that contain keywords (marc_tag = 650).~~

Filtering the data.


In [ ]:
#
# Get only those rows that has ykl- and hklj-classes
#

# Filtteröidään kaunokirjallisuus pois ("095":n luokka 1.4) ja vaaditaan etteivät 084 ja 095 ole tyhjiä.
df1 = df_raw_data[:]
df1 = df1[
    (
        (df_raw_data["084"].notnull())
        & (df_raw_data["095"].notnull())
        & (df_raw_data["095"] != "1.4")
    )
]

# 084-luokituksia (YKL) voi olla useita samalla niteelle (datariville). Valitaan ensimmäinen arvo.
df1.loc[:, ("084")] = df1["084"].apply(lambda x: x.split(",")[0])


<IPython.core.display.Javascript object>

In [ ]:
# Fixer function which scrapes "real" part from broken string.
# I.e. broken: '59.562209'  -> fixed '59.5622',
#      broken: '59.562109*' -> fixed '59.562',
#      broken: '89.41038'   -> fixed '89.41'
def fix_class(wrong_value, notations):
    # take the first value. cell could contain multiple values
    wrong_value = wrong_value.split(",")[0]
    found = False
    try:
        for index, char in enumerate(wrong_value):
            if char == ".":  # no need to test decimal point -value
                continue
            slice_of_test_value = len(wrong_value) - (index + 1)
            test_value = str(wrong_value[0:slice_of_test_value])
            if test_value in notations:
                return test_value
    except Exception as e:
        print("wrong value: ", wrong_value, " e: ", e)
        pass
    return np.nan

<IPython.core.display.Javascript object>

In [ ]:
# Function which removes "form tags" ("muotoluokka") from string.

import re


def remove_form_tags(wrong_value, tags):
    for tag in tags:
        pattern = tag + "(?!.*" + tag + ")"
        s = re.search(pattern, wrong_value)
        if s:
            position = s.start()

            # remove '.' if it the last character
            if wrong_value[position - 1] == ".":
                korjaus = wrong_value[: s.start() - 1]
            else:
                korjaus = wrong_value[: s.start()]
            return korjaus
    return np.nan

<IPython.core.display.Javascript object>

In [ ]:
# Trim (remove spaces) string values of all feature columns (084, 092, 093, 094 and 095)

df_obj = df1.select_dtypes(["object"])
df1[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())

<IPython.core.display.Javascript object>

In [ ]:
# YKL hierarchy
df_ykl = pd.read_csv(
    "data/raw_data/ykl_simple.csv",
    dtype={"notation": str, "prefLabel": str},
)

# YKL form tags
df_ykl_ml = pd.read_csv(
    "data/raw_data/ykl_ml.csv",
    dtype={"notation": str, "prefLabel": str},
)

# Read notations (unique 084-classes) into list
notations = df_ykl["notation"]
# NaNs are valid "correct" values in this syntax -> append it to notations Series
notations = notations.append(pd.Series([Nan], index=[2384]))
# Convert df to list
notations = notations.values.tolist()

# Select preferred columns for cleaning purposes(084, 092, 093 and 094)
columns = list(df1)[1:-2]

form_tags = df_ykl_ml["notation"].values.tolist()  # .wrap(width=10)  # .str.join(",")
tags = form_tags

# Create regex pattern for class-value to test if class is 'form tag'
pattern = "(?:\d{2}\.\d*"
form_tags = "|".join([pattern + "{}$)".format(value) for value in form_tags])

# Iterate over selected columns (084, 092, 093 and 094)
for i in columns:
    df_split1A = df1[df1[i].isin(notations)]
    df_split1B = df1[~df1[i].isin(notations)]
    df_split2 = df_split1B[df_split1B[i].str.fullmatch(form_tags)]

    # Remove "form tags"
    df_split2.loc[:, (i)] = df_split2.apply(
        lambda x: remove_form_tags(x[i], tags), axis=1
    )

    df_split3A = df_split2[df_split2[i].isin(notations)]
    df_split3B = df_split2[~df_split2[i].isin(notations)]

    # Fix the remaining ones
    df_split3B.loc[:, (i)] = df_split3B.apply(
        lambda x: fix_class(x[i], notations), axis=1
    )

    # Merge splitted data
    df1 = pd.concat([df_split1A, df_split3A, df_split3B], axis=0)


/anaconda/envs/ml-sami-libclas/lib/python3.8/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


<IPython.core.display.Javascript object>

In [ ]:
# Read HKLJ classes from csv-file
df_hklj = pd.read_csv(
    "data/raw_data/hklj_simple.csv",
    dtype={"notation": str, "prefLabel": str},
)

# Read HKLJ form tags from csv-file
df_hklj_ml = pd.read_csv(
    "data/raw_data/hklj_ml.csv",
    dtype={"notation": str, "prefLabel": str},
)

# Read notations (unique 084-classes) into list
notations = df_hklj["notation"]

# NaNs are valid "correct" values in this syntax -> append it to notations Series
# notations = notations.append(pd.Series([Nan], index=[2232]))
notations = notations.append(pd.Series([Nan], index=[2055]))
# Convert df to list
notations = notations.values.tolist()

# form_tags = df_hklj_ml["notation"].str.cat(sep="|")
form_tags = df_hklj_ml["notation"].values.tolist()  # .wrap(width=10)  # .str.join(",")
tags = form_tags

# Create regex pattern for class-value to test if class is 'form tag'
pattern = "(?:\d{3}\.\d*"
form_tags = "|".join([pattern + "{}$)".format(value) for value in form_tags])

# Split data to separate dataframes
df_split1A = df1[df1["095"].isin(notations)]
df_split1B = df1[~df1["095"].isin(notations)]
df_split2 = df_split1B[df_split1B["095"].str.fullmatch(form_tags)]

# Remove "form tags"
df_split2.loc[:, ("095")] = df_split2.apply(
    lambda x: remove_form_tags(x["095"], tags), axis=1
)

df_split3A = df_split2[df_split2["095"].isin(notations)]
df_split3B = df_split2[~df_split2["095"].isin(notations)]

# Fix the remaining ones
df_split3B.loc[:, ("095")] = df_split3B.apply(
    lambda x: fix_class(x["095"], notations), axis=1
)

# Merge splitted data
df1 = pd.concat([df_split1A, df_split3A, df_split3B], axis=0)


<IPython.core.display.Javascript object>

In [ ]:
# Copy dataframe
df2 = df1[:]

# Filter out rows which has empty strings (=""). You can do it by using astype(bool)-method.
df2 = df2[df2["084"].astype(bool)]

# Filter again null values
df2 = df2[((df2["084"].notnull()) & (df2["095"].notnull()))]

# Write dataframe to csv-file
df2.to_csv("data_output.csv")

<IPython.core.display.Javascript object>

In [ ]:
# Create schema and validate values against that schema.

import pandas_schema
from pandas_schema import Column, Schema
from pandas_schema.validation import (
    CustomElementValidation,
    LeadingWhitespaceValidation,
    TrailingWhitespaceValidation,
    CanConvertValidation,
    MatchesPatternValidation,
    InRangeValidation,
    InListValidation,
)

null_validation = [
    CustomElementValidation(lambda d: d is not np.nan, "this field cannot be null")
]

schema = Schema(
    [
        Column("record_id", null_validation),
        Column(
            "084",
            [
                LeadingWhitespaceValidation(),
                TrailingWhitespaceValidation(),
                MatchesPatternValidation(r"^(?:\S)|\d{1,2}(?:$|.\d{0,}$)"),
            ],
        ),
        Column("092", [LeadingWhitespaceValidation(), TrailingWhitespaceValidation()]),
        Column("093", [LeadingWhitespaceValidation(), TrailingWhitespaceValidation()]),
        Column("094", [LeadingWhitespaceValidation(), TrailingWhitespaceValidation()]),
        Column("095", [LeadingWhitespaceValidation(), TrailingWhitespaceValidation()]),
        Column("650", [LeadingWhitespaceValidation(), TrailingWhitespaceValidation()]),
    ]
)

errors = schema.validate(df2)

# Count of validation errors
print('Count of errors:', len(errors))

# Print first 15 validation errros
for error in errors[:15]:
    print(error)


Count of errors: 0


<IPython.core.display.Javascript object>

The amount of library items and classes is big (hundreds of thousands items and thousands of classes). To limit the problem we make the following assumptions:
 - We always know the YKL-classification of the new library item, that we want classify with HKL-classification sytem.
 - Training/validation set can and need only contain rows that match the YKL-classification of the new library item.
 
Thus we pick only those rows that match the YKL-class of the new library item. In first implementation just select a random YKL-class. However the training/validation dataset can still be quite big. You can restrict the sixe of the dataset even more by setting the variable MAX_TRAININGSET_SIZE below appropriately.

In [ ]:
# Restrict the trainingset size if necessary
MAX_TRAININGSET_SIZE = 3000

# Select a random YKL-class get all rows matching it
sample_row = df2.sample(frac=1)
item_ykl_class = sample_row.iloc[0]["084"]
one_ykl_class_df = df2.loc[df2["084"] == item_ykl_class]
one_ykl_class_df = one_ykl_class_df.reset_index(drop=True)

# Read all distinct keywords into an array
keywords = []
for i in range(len(one_ykl_class_df)):
    item_keywords_str = (str)(one_ykl_class_df.iloc[(i), 6])
    item_keywords_lst = item_keywords_str.split(",")
    for word in item_keywords_lst:
        word = word.strip().lower()
        if word not in keywords:
            keywords.append(word)

# Sanity check printing
print(f"\n*****")
print(f"Found {one_ykl_class_df.shape[0]} rows with YKL-CLASS={item_ykl_class}")
print(f"Number of keywords: {len(keywords)}")
print(f"*****\n")

# Restrict the training/validation set MAX_TRAININGSET_SIZE rows
if one_ykl_class_df.shape[0] < MAX_TRAININGSET_SIZE:
    MAX_TRAININGSET_SIZE = one_ykl_class_df.shape[0]
one_ykl_class_df = one_ykl_class_df.sample(MAX_TRAININGSET_SIZE)

# Add keyword columns with keyword as a title (value will be 0 or 1 depending if the keyword belongs to the volume or not)
# NOTE: "A Pandas Series is like a column in a table" (https://www.w3schools.com/python/pandas/pandas_series.asp)
for i in range(len(keywords)):
    one_ykl_class_df[keywords[i]] = pd.Series([], dtype="int64")
one_ykl_class_df = one_ykl_class_df.reset_index(drop=True)

# Create
for i in range(len(one_ykl_class_df)):
    item_keywords_str = (str)(one_ykl_class_df.iloc[(i), 6])
    item_keywords_lst = item_keywords_str.split(",")
    for word in item_keywords_lst:
        word = word.strip().lower()
        one_ykl_class_df.at[i, word] = 1

# Drop column with comma-separated keywords and fill NaN with 0
one_ykl_class_df = one_ykl_class_df.drop(["650"], axis=1)
one_ykl_class_df = one_ykl_class_df.fillna(0)

# Sanity check: printing final dataset for training and validating the model
print(f"\n*****")
print(
    f"Final dataset: {one_ykl_class_df.shape[0]} rows with YKL-CLASS={item_ykl_class}"
)
print(f"Number of keywords in final dataset is: {one_ykl_class_df.shape[1] - 6}")
print(f"*****\n")
one_ykl_class_df.head()


*****
Found 3107 rows with YKL-CLASS=78.8911
Number of keywords: 365
*****


*****
Final dataset: 3000 rows with YKL-CLASS=78.8911
Number of keywords in final dataset is: 365
*****



,record_id,084,092,093,094,095,rock,popmusiikki,kokoelmat,tabulatuurinuottikirjoitus,...,heavy metal,säveltäminen,huuliharppu,rockfestivaalit,ruisrock,ilosaarirock,tapahtumat,elävä musiikki,tyylit,hakuteokset
0,420909247702,78.8911,78.8911,0,0,786.231,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,420908439649,78.8911,78.8911,0,0,788.33,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,420908552378,78.8911,78.8911,0,78.8911,788.33,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,420909189008,78.8911,78.8911,78.8911,0,788.33,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,420908424085,78.8911,78.8911,0,78.8911,788.33,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


<IPython.core.display.Javascript object>

***

<div class="alert alert-block alert-danger">

    **HUOM!**

    Template on testattu tähän soluun asti. Loppu, lukuunottamatta lopussa olevia soluja on vielä siivoamatta/refaktoroimatta.

    DataFrame jota voi käyttää tämän solun jälkeen: "df2" tai "one_ykl_class_df", joka on lopullinen Training/Validation datasetti, ja jossa on mukana vain yksi yhden YKL luokan rivit.
    
    Notebookin lopussa on siis toimivia ja  testattuja soluja, joissa vielä viimeitellään lopullinen dataset ja toy dataset ja tallennetaan ne csv-muodossa tiedostoon. 

</div>

***

In [ ]:
# DO NOT USE THIS CELL - IT IS NOT WORKING!!!!!

##df2 = df1[:]
# df2 = df_cut[:]
# df2 = df_uncut[:]
df2 = df_F[:]
# df2['084'] = pd.to_numeric(df2["084"], errors='coerce')
# df2['084'] = df2['084'].astype(str)
c1 = df2.sort_values(by=["084"])
c2 = c1["084"].unique()
# c1.sort_values()
# """
print(len(c2))
# print(c2)
# """

print("NV: ", notations.values)
"""
for row in c2[:10]:
    # print(type(row))
    if row not in notations.values:
        print(row)
"""

c3 = c1[~c1["084"].isin(notations)]

print(c3["084"].isnull().sum())

measurer = np.vectorize(len)
res1 = measurer(df2.values.astype(str)).max(axis=0)
print("Measures: ", res1)

df2 = df2[(df2["084"].notnull())]

df2["084_length"] = df2["084"].str.len()
# s = df2["084"].str.len().sort_values()
# df2.sort_values("084_length", ascending=True, inplace=True)
# print(df2.head(2000))

luokat_lkm = df2["084"].value_counts().nlargest(10)
print(luokat_lkm)
luokat_lkm.plot(kind="bar")

"""
# s_temp = luokat_lkm
s_temp = df2["084_length"].sort_values(ascending=False)

# Count how many rows are not in the top ten
not_top_ten = len(s_temp) - 10
print(not_top_ten)

# Sum the values not in the top ten
not_top_ten_sum = s_temp.tail(not_top_ten).sum()
print(not_top_ten_sum)

# Get the top ten values
s_top = s_temp.head(10)
# s_top = s_temp[:10]
print(s_top)

# Append the sum of not-top-ten values to the Series
s_top[10] = not_top_ten_sum
# s_top.append(not_top_ten_sum)

# Plot pie chart
_ = s_top.plot.pie()

# Show plot
plt.show()
"""


# hist = df2.hist(column="084_length", bins=7)
# hist = luokat_lkm.hist()
# plot = luokat_lkm.plot.pie()

# df2.to_csv("output.csv")

In [ ]:
# Read all distinct keywoards into an array
keywords = []
# for i in range(len(df1[:10])):
for index, row in df2.iterrows():
    # print(i)
    # k = df1.loc[i+1, "650"]
    try:
        kwords = row["650"].split(",")
    except Exception as e:
        # print(row["650"])
        kwords = None
    if kwords:
        for k in kwords:
            if k not in keywords:
                keywords.append(k)
        # print(index, row["650"])
    # if index > 500000:
    #    break
print("---")
# print(keywords)
print(len(keywords))

---
31032


<IPython.core.display.Javascript object>

Here, all columns that contain 'object' variables, contain elements that can not be described in sigle datatype.
These may be nested structures or mixed datatypes. We saw earlier that there were a lot of question marks in the dataset.
Let's try to replace these with np.nan, meaning a missing value, and then converting the columns with 'object' datatype to numericals:

In [ ]:
# replace '?' with np nan
df.replace("?", np.nan, inplace=True)

In [ ]:
# convert columns with 'object' type to float
df = df.astype(
    {
        "x4trestbps": float,
        "x6fbs": float,
        "x7restecg": float,
        "x8thalach": float,
        "x9exang": float,
        "x10oldpeak": float,
        "x11slope": float,
        "x12ca": float,
        "x13thal": float,
    }
)
df.dtypes

Ok, now there are no 'object' datatypes - if there were, we would know that there are still some malicius values in the cells that would have to be cleaned off.

For the sake of simplicity, we want to omit all categorical features in this example.

In [ ]:
df.nunique()

We see that the features "x2sex", "x3cp", "x5chol", "x7restecg" and "x9exang" only contain limited number of values,
 so we assume them categorical. These, we can drop from the data.

In [ ]:
# drop columns by name
df = df.drop(
    ["x2sex", "x3cp", "x5chol", "x7restecg", "x9exang"],
    axis=1,  # drop columns with categorical variables
)

Let us see how much of the data is missing in each variable left

In [ ]:
# count nan values per column
df.isna().sum()

Ok,  there are many missing variables.

Now we have two alternatives: we can either get rid of the missing values, or use such robust methods that allow missing values in the data.

For removing the missing values we can either drop the columns with most missing data or try to impute the missing values from the data.

If there was very little data, we would most likely want to impute the missing data,
for example with use the Scikit-learn imputer: https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.
If you do imputing of missing data, please consider the effects on data quality.

In this example, we just drop the columns with most na data





In [ ]:
df = (
    df.drop(
        ["x6fbs", "x11slope", "x12ca", "x13thal"],  # drop columns with most na values
        axis=1,
    )
    .dropna()
    .reset_index(drop=True)
)

In [ ]:
df.head()

## Visualize the data

Once the dataset is looking clean, it is time to visualize it.
Visualization is probably the most powerful tool of data science.

Visualization depends highly on the data, but usually you should begin by looking at two things: distribution and correlation.

Let's make histograms of the variables, and a trellis of scatterplots for visualizing correlations.

In [ ]:
# export

# ^(this cell is exported to ml_project_template/ml_project_template/data.py)
# you could also define another module to export to.
# however, all modules that you export to, must have a notebook with the same name and header!

# function for drawing histograms of a dataframe
import matplotlib.pyplot as plt
import pandas as pd


def plot_histogram(df):
    """
    Plot histograms of a dataframe
    """
    n_cols = len(df.columns)
    col_names = df.columns.values
    fig, axs = plt.subplots(
        1, n_cols, figsize=(3 * n_cols, 3), constrained_layout=True, sharey="row"
    )
    for i in range(n_cols):
        ax = axs[i]
        col_name = col_names[i]
        x = df[col_name]
        x.plot(ax=ax, kind="hist")
        xmin, xmax = min(x), max(x)
    for i in range(n_cols):  # to have equal tick lines in each plot
        ax = axs[i]
        col_name = col_names[i]
        x = df[col_name]
        xmin, xmax = min(x), max(x)
        ax.hlines(
            y=axs[df.apply(lambda x: x.max()).argmin()].get_yticks()[
                1:-1
            ],  # select ticks from the fig with smallest max value
            xmin=xmin,
            xmax=xmax,
            colors="white",
            alpha=1,
            linewidth=2,
        )
        ax.set_xlabel(col_name)
        ax.set_title(f"{i})", loc="left")
        ax.spines["left"].set_visible(False)
        ax.spines["right"].set_visible(False)
        ax.spines["top"].set_visible(False)
    return fig, ax

Plot histogram of the data:

In [ ]:
_ = plot_histogram(df)

From the plot 4) we see the distribution of different heart disease diagnosis.
Remember, that the diagnosis 0 is negative, meaning no heart disease, and the rest are positive, meaning different heart diagnoses.
For sake of simplicity, we will replace all different positive diagnoses with a one positive indicator.

Let's replace different heart diagnoses with value 1

In [ ]:
df.y1num.replace([1, 2, 3, 4], 1, inplace=True)

Let's plot the distributions again:

In [ ]:
_ = plot_histogram(df)

Let's make a trellis plot of the features. Now we can visualize the diagnosis result with the glyph (color and shape of the marker).

But wait, don't we just want to plot each of the features against the label?
You could do that, too, but in the initial analysis phase it is a good practice to compare all variables against each other.
It can reveal all kinds of interesting correlations that may affect your choises later.

This will make quite a plot to digest, I agree.
For a raport or a presentation you might want to focus on just a few features.

In [ ]:
# export

# Make a multiple of scatter plots
def plot_trellis(df, legend_title="y", true_label="True", false_label="False"):
    """
    Make a trellis plot of a dataframe against a binary y value in last column
    """
    n_cols = df.shape[1] - 1
    col_names = df.columns.values

    fig, axs = plt.subplots(
        n_cols, n_cols, figsize=(n_cols * 3, n_cols * 3), constrained_layout=True
    )
    for i in range(n_cols):
        for j in range(n_cols):
            ax = axs[i, j]
            if i != j:
                # first plot negative cases
                df[df.iloc[:, -1] == 0].plot(
                    ax=ax,
                    x=col_names[j],
                    y=col_names[i],
                    kind="scatter",
                    color="b",
                    marker="o",
                    alpha=0.5,
                    label=false_label,
                )
                # then positive cases
                df[df.iloc[:, -1] != 0].plot(
                    ax=ax,
                    x=col_names[j],
                    y=col_names[i],
                    kind="scatter",
                    color="r",
                    marker="x",
                    alpha=0.5,
                    label=true_label,
                )
                # Hide the right and top spines
                ax.spines["right"].set_visible(False)
                ax.spines["top"].set_visible(False)
                ax.set_title(f"{i*n_cols+j})", loc="left")
                ax.legend(title=legend_title)
            else:
                ax.annotate(xy=(0, 0.5), text=col_names[i], fontsize=20)
                ax.axis("off")  # hide the box
    return fig, ax

In [ ]:
_ = plot_trellis(
    df, legend_title="diagnose", true_label="positive", false_label="negative"
)

***

<div class="alert alert-block alert-danger">

    **HUOM!**

    Tästä eteenpäin taas toimivaa koodia. Viimeistellään lopullinen training/validation datasetti ja toy datasetti ja tallennetaan sisältö seuraaviin tiedostoihin:
    - "data/preprocessed_data/dataset_clean_hki_lib_book_classification.csv"
    - "data/preprocessed_data/dataset_toy_hki_lib_book_classification.csv"
</div>

***

## Some intermediate conclusions based on data visualization

We are interested in science-classifications mostly (how to separate them?).

The trellis plotting of features can probably be interpreted so that there is very strong correlation between classifications.

NOTE: We can also see that there are some outliers. We should find these values and maybe get rid of them (perhaps replace them with NaN or zero). Or is it possible, that these outliers are the ones that ML-algorithm classification is of most help?

 - [Outliers with scikit-learn](http://napitupulu-jon.appspot.com/posts/outliers-ud120.html)
 - [How should outliers be dealt with in linear regression analysis?](https://stats.stackexchange.com/questions/175/how-should-outliers-be-dealt-with-in-linear-regression-analysis)
 - [What is the difference between linearly dependent and linearly correlated?](https://stats.stackexchange.com/questions/31270/what-is-the-difference-between-linearly-dependent-and-linearly-correlated)

## Suffle Dataset

Before saving the dataset for further use, it is a good practice to suffle it.
This is, of course, assuming the order of the data is not meaningful (as with for example time series data).

Random samplin large datasets consumes time and resources, so if you have a pre-suffled dataset you can just read in more data that is already randomized.
Also, quite often datasets are intentionally or carelessly saved with some obvious or latent order that might include odd biases to your further analysis.
Suffling helps you to get rid of these. Suffling a large dataset may take a while.

In [ ]:
# to suffle the dataset sample a fraction of 1 from it
one_ykl_class_df = one_ykl_class_df.sample(frac=1).reset_index(
    drop=True
)  # suffle and re-index dataset
one_ykl_class_df.head()

,record_id,084,092,093,094,095,rock,popmusiikki,kokoelmat,tabulatuurinuottikirjoitus,...,heavy metal,säveltäminen,huuliharppu,rockfestivaalit,ruisrock,ilosaarirock,tapahtumat,elävä musiikki,tyylit,hakuteokset
0,420908696117,78.8911,0,0,0,788.33,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,420908412353,78.8911,78.8911,0,0,788.335,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,420907962293,78.8911,78.8911,78.8911,78.8911,788.33,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,420908458859,78.8911,78.8911,0,78.8911,788.33,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,420909057565,78.8911,0,0,78.8911,788.335,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


<IPython.core.display.Javascript object>

## Save clean and tidy data for further use

In [ ]:
# save the dataset to output path defined in the beginning of this notebook
one_ykl_class_df.to_csv(output_clean_filepath)

<IPython.core.display.Javascript object>

## Create small toy dataset for developing and testing the ML methods

Best way to test your ML methods in development is with data as close to the real data as possible.
However, to save you from frustrating long runtimes in development, it is better to create a small sample dataset.
You can also create unit tests with this sample dataset.

In [ ]:
# create simple stratified sample of the notebook
n_toy = 1000  # number of samples
n_toy = min(one_ykl_class_df.shape[0], n_toy)

# toy_df = (
#    one_ykl_class_df.groupby("record_id", group_keys=False)
#    .apply(
#        lambda x: x.sample(int(np.rint(n_toy * x.shape[0] / one_ykl_class_df.shape[0])))
#    )
#    .sample(frac=1)
#    .reset_index(drop=True)
# )

# alternatively:
toy_df = one_ykl_class_df.sample(
    frac=n_toy / one_ykl_class_df.shape[0]
)  # if simple stratification is not applicable
# toy_df = df.iloc[:n_toy,:] # with time series data. consider the size of n_toy


print(f"\n*****")
print(f"Toy dataset size: {toy_df.shape[0]} rows and {toy_df.shape[1]} columns")
print(f"*****\n")
toy_df.head()


*****
Toy dataset size: 1000 rows and 371 columns
*****



,record_id,084,092,093,094,095,rock,popmusiikki,kokoelmat,tabulatuurinuottikirjoitus,...,heavy metal,säveltäminen,huuliharppu,rockfestivaalit,ruisrock,ilosaarirock,tapahtumat,elävä musiikki,tyylit,hakuteokset
1396,420909179575,78.8911,78.8911,0,78.8911,788.335,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
349,420908618411,78.8911,78.8911,0,0,788.33,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1378,420908728550,78.8911,78.89113,0,0,788.334,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
630,420908846341,78.8911,78.8911,0,0,788.33,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2735,420908440456,78.8911,0,0,0,788.33,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


<IPython.core.display.Javascript object>

In [ ]:
# save toy dataset
toy_df.to_csv(output_toy_filepath)

<IPython.core.display.Javascript object>

## You can now move on to the model notebook!